### Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyperparameter

In [ ]:
batch_size = 25
reduction_dim = 30
lr = 0.002

### Get Data

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5], [0.5]
        )
    ]
)

folder_path = "data"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# create datasets
dataset_train = datasets.MNIST(root=folder_path, train=True, transform=transform, download=True)
dataset_test = datasets.MNIST(root=folder_path, train=False, transform=transform, download=False)

# create dataloaders
loader_train = torch.utils.data.Dataloader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
loader_test = torch.utils.data.Dataloader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

### Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, e_input_dim, e_output_dim):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(e_input_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, e_output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

### Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d_input_dim, d_output_dim):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(d_input_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, d_output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

### Network

In [ ]:
data_dim = 1
# automatically calculate the dimension
for dimension in range(1, dataset_train.data.ndim):
    data_dim *= dataset_train.data.size(dimension)

E = Encoder(e_input_dim= data_dim, g_output_dim=reduction_dim).to(device)
D = Decoder(d_input_dim= reduction_dim, d_output_dim=data_dim).to(device)

lossFunction = nn.BCELoss()

# Optimizers

Gen_optimizer = optim.Adam(E.parameters(), lr=lr)
Dis_optimizer = optim.Adam(D.parameters(), lr=lr)